# Dependencies setup

In [ ]:
import warnings
try:
    import torch
except ImportError:
    %pip install torch
    import torch
warnings.filterwarnings('ignore')
#robustbench download
%pip install -q git+https://github.com/RobustBench/robustbench.git
#autoattack download
%pip install -q git+https://github.com/fra31/auto-attack
from autoattack import AutoAttack
if torch.cuda.is_available():
    device = "cuda"
else:
    device ="cpu"

models = ["Addepalli2022Efficient_RN18","Debenedetti2022Light_XCiT-S12","Debenedetti2022Light_XCiT-M12",
          "Debenedetti2022Light_XCiT-L12","Cui2023Decoupled_WRN-28-10","Addepalli2022Efficient_WRN_34_10",
          "Huang2022Revisiting_WRN-A4","Wang2023Better_WRN-70-16","Rebuffi2021Fixing_106_16_cutmix_ddpm"]
attacks_to_run=['apgd-ce','apgd-dlr','fab-t']


# Acquiring the datasets and the models from RobustBench

In [ ]:
from robustbench.data import load_cifar10
x_test,y_test = load_cifar10(n_examples=500)
from robustbench.utils import load_model
for m in models:
    model = load_model(model_name=m,threat_model='Linf')

# Overall and per class accuracy evaluation

In [ ]:
for m in models:
    torch.cuda.empty_cache()
    print("Starting tests for model: {}".format(m))
    model = load_model(model_name=m, threat_model="Linf")
    model.to(device)
    x_test.to(device)
    y_test.to(device)
    num_class = 10
    for i in range(num_class):
        class_indexes = y_test == i
        x_class = x_test[class_indexes].to(device)
        y_class = y_test[class_indexes].to(device)
        print("data for class {}, len of y {}".format(i, len(y_class)))

    # overall accuracy evaluation
    adversary = AutoAttack(
        model,
        norm="Linf",
        eps=8 / 255,
        version="custom",
        attacks_to_run=attacks_to_run,
        device=device,
    )
    adversary.apgd.n_restarts = 1
    x_adv = adversary.run_standard_evaluation(x_test, y_test)

    # accuracy evaluation per class
    new_adversary = AutoAttack(
        model,
        norm="Linf",
        eps=8 / 255,
        version="custom",
        attacks_to_run=attacks_to_run,
        device=device,
    )
    new_adversary.apgd.n_restarts = 1
    for i in range(0, num_class):
        torch.cuda.empty_cache()
        class_indexes = y_test == i
        x_class = x_test[class_indexes].to(device)
        y_class = y_test[class_indexes].to(device)
        print("Robustness for class {}".format(i))
        x_adv = new_adversary.run_standard_evaluation(x_class, y_class)
        print("#####################")
    print("Finished tests for model: {}".format(m))